# 환경설치

In [1]:
!pip install html5lib beautifulsoup4
!pip install selenium

from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import requests
import json
import re
import pandas as pd

In [2]:
from urllib import request
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException

In [3]:
from selenium.webdriver.common.keys import Keys

In [4]:
import numpy as np

# 아이돌별 멜론 아이디가 지정된 파일 읽기

In [5]:
# 인코딩이 euc-kr입니다. 저걸 지정하지 않으면 csv가 안 열려요!
# idol_id가 저장된 csv 파일을 불러오셔야 합니다. 멜론 url에 가수를 찾기 위한 id입니다. 

idol_list = pd.read_csv('1996_1997_debut.csv', encoding='euc-kr')
idol_list.head()

,artist,artist_eng,member_num,leader_name,gender,agency,idol_id
0,언타이틀,Untitle,2,유건형,남,레오뮤직,100042
1,Hamohamo,HAMO HAMO,2,이성진,남,뮤직 팩토리,100178
2,아이돌,IDOL,2,최혁준,남,DSP 미디어,101640
3,구피,Goofy,3,이승광,남,비온디크루,100023
4,태사자,Taesaja,4,김형준,남,매스컴,100096


In [6]:
idol_list.shape

(10, 7)

In [7]:
idol_list = idol_list[np.isfinite(idol_list['idol_id'])].reset_index(drop=True)

In [8]:
idol_list.shape

(10, 7)

In [9]:
# id가 flaot으로 저장되어 있습니다. int로 바꿔주고 저장하겠습니다. 
idol_list['idol_id'] = idol_list['idol_id'].astype(int)
idol_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   artist       10 non-null     object
 1   artist_eng   10 non-null     object
 2   member_num   10 non-null     object
 3   leader_name  9 non-null      object
 4   gender       10 non-null     object
 5   agency       10 non-null     object
 6   idol_id      10 non-null     int64 
dtypes: int64(1), object(6)
memory usage: 688.0+ bytes


In [10]:
# 수집을 위한 빈 리스트 생성

title = []
release_date = []
song_genre = []
lyrics = []
creator = []
artist = []
artist_m = []
debut_y = []
debut_t = []
agency = []

In [61]:
# 곡마다 가수 이름을 붙여주어야 하기에 가수 이름 리스트로 받아오기
names = idol_list.artist
names[0]

'언타이틀'

In [12]:
# 크롤러가 링크를 찾을 때 매번 바꿔주는 값. for문에 사용됩니다. 
ids = idol_list.idol_id
ids[0]

100042

In [13]:
# tests = ['780066']

# 크롤러
### 아이돌별 96~99년도 발매한 곡만 수집
: 수집 가능 목록 (발매날짜/제목/가사/작사/작곡)

In [14]:
path = "/Users/mac/opt/anaconda3/bin/chromedriver"
driver = webdriver.Chrome(path)
driver.get("https://www.melon.com")

In [15]:
# 곡 정보를 찾아들어가기 위한 url을 분해해서 변수 지정해놓은 모습입니다. 
# l3 뒤에 붙는 숫자가 멜론 서버에 저장된 해당 가수의 해당 곡 번호입니다. 

l1 = 'https://www.melon.com/artist/song.htm?artistId='
# idol_id
l2 = '#params%5BlistType%5D=A&params%5BorderBy%5D=ISSUE_DATE&params%5BartistId%5D='
# idol_id
l3 = '&po=pageObj&startIndex='
# l1 + 'idol_id' + l2 + 'idol_id' + l3 + number

# 가수 이름을 넣기 위한 변수
i = 0
tmp = []
str_tmp = ""
arr = []

for id in ids:
    
    n = 1
    list_n = 1
    while True:
    #for k in range(1):
                 
        
        print('n: ',n)
        driver.get(l1+str(id)+l2+str(id)+l3+str(n))    
        sleep(2)
        
                
        # 곡 정보로 들어가기
        try:
            driver.find_element_by_xpath('//*[@id="conts"]/div[3]/ul/li[3]/a').click()
            str_tmp = '//*[@id="frm"]/div/table/tbody/tr[' + str(list_n) + ']/td[3]/div/div/a[1]'
            driver.find_element_by_xpath(str_tmp).click()
            print('list: ',list_n)
            # 발매날짜 크롤링
            try:
                str_tmp = driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[2]').text
                arr = str_tmp.split('.')
                print("check :",str_tmp)
                if( int(arr[0]) < 2000 and int(arr[0]) > 1995): # 96,97,98,99 발매 노래만
                    print("input: ", str_tmp)
                    release_date.append(str_tmp)
                    list_n += 1
                elif (int(arr[0]) < 1996): # 96년 이전 발매일 경우 break
                    break
                else:
                    list_n += 1
                    continue
                
            
            except NoSuchElementException as e:
                release_date.append('')
            
            # 제목 크롤링(제목은 없지 않기 때문에 예외 없음)
            title.append(driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[1]/div[1]').text)
            print('title:', title)

            try:
                song_genre.append(driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[3]').text)
            except NoSuchElementException as e:
                song_genre.append('')

            # 가사 크롤링
            try:
                lyrics.append(driver.find_element_by_xpath('//*[@id="d_video_summary"]').text)
            except NoSuchElementException as e:
                lyrics.append('')

            # 작사 작곡 내역이 없는 경우
            try:
                creator.append(driver.find_element_by_xpath('//*[@id="conts"]/div[3]/ul').text)
            except NoSuchElementException as e:
                creator.append('')

            n += 1
            
            # 가수 이름 붙이기
            artist.append(names[i])
            
            # 디버깅    
            print(len(title), title[len(title)-1])

        except NoSuchElementException as e:
            i += 1
            break
        
        

n:  1
list:  1
check : 2007.12.24
n:  1
list:  2
check : 2007.12.13
n:  1
list:  3
check : 2003.11.01
n:  1
list:  4
check : 2000.11.01
n:  1
list:  5
check : 1999.08.01
input:  1999.08.01
title: ['떠나가지 마세요']
1 떠나가지 마세요
n:  2
list:  6
check : 1999.03.05
input:  1999.03.05
title: ['떠나가지 마세요', '학교 (Main Title)']
2 학교 (Main Title)
n:  3
list:  7
check : 1999.02.01
input:  1999.02.01
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요']
3 떠나가지 마세요
n:  4
list:  8
check : 1999.02.01
input:  1999.02.01
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기']
4 나의 슬픈 사랑이야기
n:  5
list:  9
check : 1999.02.01
input:  1999.02.01
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기', '사랑연가']
5 사랑연가
n:  6
list:  10
check : 1999.02.01
input:  1999.02.01
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기', '사랑연가', '댄스! 댄스! 댄스!']
6 댄스! 댄스! 댄스!
n:  7
list:  11
check : 1999.02.01
input:  1999.02.01
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기', '사랑연가', '댄스

list:  8
check : 1996.11
input:  1996.11
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기', '사랑연가', '댄스! 댄스! 댄스!', '하얀 사랑', '낮과 밤', '인연', '검은 위선', 'Love Song #2', 'Run', 'Play Boy', 'Goodbye', '우리가 시작할때...거기엔 랩이 있었다', 'Intro', '꽃', '승리', '눈 내리는 날', 'Don`t Stop The Music', '불', '정.경', '왜', '빈자리', '자살', 'Pang Pang', '빠삐용 (그의 두통은 자유였다)', '마라! 봐라!', '아니벌써', 'Silver Line', 'True Love', '반쪽만의 사랑 (My Teacher)', '히든 카드']
33 히든 카드
n:  9
list:  9
check : 1996.11
input:  1996.11
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기', '사랑연가', '댄스! 댄스! 댄스!', '하얀 사랑', '낮과 밤', '인연', '검은 위선', 'Love Song #2', 'Run', 'Play Boy', 'Goodbye', '우리가 시작할때...거기엔 랩이 있었다', 'Intro', '꽃', '승리', '눈 내리는 날', 'Don`t Stop The Music', '불', '정.경', '왜', '빈자리', '자살', 'Pang Pang', '빠삐용 (그의 두통은 자유였다)', '마라! 봐라!', '아니벌써', 'Silver Line', 'True Love', '반쪽만의 사랑 (My Teacher)', '히든 카드', '세상에 던지는 한마디']
34 세상에 던지는 한마디
n:  10
list:  10
check : 1996.11
input:  1996.11
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나

list:  12
check : 1996.02
input:  1996.02
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기', '사랑연가', '댄스! 댄스! 댄스!', '하얀 사랑', '낮과 밤', '인연', '검은 위선', 'Love Song #2', 'Run', 'Play Boy', 'Goodbye', '우리가 시작할때...거기엔 랩이 있었다', 'Intro', '꽃', '승리', '눈 내리는 날', 'Don`t Stop The Music', '불', '정.경', '왜', '빈자리', '자살', 'Pang Pang', '빠삐용 (그의 두통은 자유였다)', '마라! 봐라!', '아니벌써', 'Silver Line', 'True Love', '반쪽만의 사랑 (My Teacher)', '히든 카드', '세상에 던지는 한마디', 'Pang Pang (나보다 나은 너)', '빠삐용 (그의 두통은 자유였다) (Hamo Ver.)', 'Intro', 'Be My Guest', '환상체험', '건망증', '이럴수가', 'Love Story', '통화중 (감기)', '삼인사각', '어쨌건...', '도사 (Jungle Trance)', '귀여운 악마', 'Bow Wow']
48 Bow Wow
n:  13
list:  13
check : 1996.02
input:  1996.02
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기', '사랑연가', '댄스! 댄스! 댄스!', '하얀 사랑', '낮과 밤', '인연', '검은 위선', 'Love Song #2', 'Run', 'Play Boy', 'Goodbye', '우리가 시작할때...거기엔 랩이 있었다', 'Intro', '꽃', '승리', '눈 내리는 날', 'Don`t Stop The Music', '불', '정.경', '왜', '빈자리', '자살', 'Pang Pang', '빠삐용 (그의 두통

list:  49
check : 1999.07
input:  1999.07
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기', '사랑연가', '댄스! 댄스! 댄스!', '하얀 사랑', '낮과 밤', '인연', '검은 위선', 'Love Song #2', 'Run', 'Play Boy', 'Goodbye', '우리가 시작할때...거기엔 랩이 있었다', 'Intro', '꽃', '승리', '눈 내리는 날', 'Don`t Stop The Music', '불', '정.경', '왜', '빈자리', '자살', 'Pang Pang', '빠삐용 (그의 두통은 자유였다)', '마라! 봐라!', '아니벌써', 'Silver Line', 'True Love', '반쪽만의 사랑 (My Teacher)', '히든 카드', '세상에 던지는 한마디', 'Pang Pang (나보다 나은 너)', '빠삐용 (그의 두통은 자유였다) (Hamo Ver.)', 'Intro', 'Be My Guest', '환상체험', '건망증', '이럴수가', 'Love Story', '통화중 (감기)', '삼인사각', '어쨌건...', '도사 (Jungle Trance)', '귀여운 악마', 'Bow Wow', 'Home Boy', 'Street Boy', '둘만의 사랑', '꿈속의 그녀', 'Memory', 'Techno Time', 'Magic']
55 Magic
n:  4
list:  50
check : 1999.07
input:  1999.07
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기', '사랑연가', '댄스! 댄스! 댄스!', '하얀 사랑', '낮과 밤', '인연', '검은 위선', 'Love Song #2', 'Run', 'Play Boy', 'Goodbye', '우리가 시작할때...거기엔 랩이 있었다', 'Intro', '꽃', '승리', '눈 내리는 날', '

list:  9
check : 1998.06
input:  1998.06
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기', '사랑연가', '댄스! 댄스! 댄스!', '하얀 사랑', '낮과 밤', '인연', '검은 위선', 'Love Song #2', 'Run', 'Play Boy', 'Goodbye', '우리가 시작할때...거기엔 랩이 있었다', 'Intro', '꽃', '승리', '눈 내리는 날', 'Don`t Stop The Music', '불', '정.경', '왜', '빈자리', '자살', 'Pang Pang', '빠삐용 (그의 두통은 자유였다)', '마라! 봐라!', '아니벌써', 'Silver Line', 'True Love', '반쪽만의 사랑 (My Teacher)', '히든 카드', '세상에 던지는 한마디', 'Pang Pang (나보다 나은 너)', '빠삐용 (그의 두통은 자유였다) (Hamo Ver.)', 'Intro', 'Be My Guest', '환상체험', '건망증', '이럴수가', 'Love Story', '통화중 (감기)', '삼인사각', '어쨌건...', '도사 (Jungle Trance)', '귀여운 악마', 'Bow Wow', 'Home Boy', 'Street Boy', '둘만의 사랑', '꿈속의 그녀', 'Memory', 'Techno Time', 'Magic', 'Vanessa', '그대야 미안해', '와요와요', 'Intro', '일곱개의 행복 Ⅰ', '사랑의 Dash', '경제공황 (I.M.F)', '외톨박이 (Face To Face)']
63 외톨박이 (Face To Face)
n:  6
list:  10
check : 1998.06
input:  1998.06
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기', '사랑연가', '댄스! 댄스! 댄스!', '하얀 사랑', '낮과 밤', '인

list:  19
check : 1997.07.01
input:  1997.07.01
title: ['떠나가지 마세요', '학교 (Main Title)', '떠나가지 마세요', '나의 슬픈 사랑이야기', '사랑연가', '댄스! 댄스! 댄스!', '하얀 사랑', '낮과 밤', '인연', '검은 위선', 'Love Song #2', 'Run', 'Play Boy', 'Goodbye', '우리가 시작할때...거기엔 랩이 있었다', 'Intro', '꽃', '승리', '눈 내리는 날', 'Don`t Stop The Music', '불', '정.경', '왜', '빈자리', '자살', 'Pang Pang', '빠삐용 (그의 두통은 자유였다)', '마라! 봐라!', '아니벌써', 'Silver Line', 'True Love', '반쪽만의 사랑 (My Teacher)', '히든 카드', '세상에 던지는 한마디', 'Pang Pang (나보다 나은 너)', '빠삐용 (그의 두통은 자유였다) (Hamo Ver.)', 'Intro', 'Be My Guest', '환상체험', '건망증', '이럴수가', 'Love Story', '통화중 (감기)', '삼인사각', '어쨌건...', '도사 (Jungle Trance)', '귀여운 악마', 'Bow Wow', 'Home Boy', 'Street Boy', '둘만의 사랑', '꿈속의 그녀', 'Memory', 'Techno Time', 'Magic', 'Vanessa', '그대야 미안해', '와요와요', 'Intro', '일곱개의 행복 Ⅰ', '사랑의 Dash', '경제공황 (I.M.F)', '외톨박이 (Face To Face)', '아름다운 이별 Part 1', '남자가 여자를 사랑할때', '진달래꽃', '너와 함께 할꺼야', '단심가', '좋은 세상 만들기 (하나)', '일곱개의 행복 Ⅱ', '하나 + 일 = ?', 'Esmeralda', '영화같은 이별']
73 영화같은 이별
n:  16
list:  20
check : 1997.

## Dataframe에 크롤링된 데이터 저장하기
### 이전 csv파일에서 남녀, 소속사를 크롤링한 데이터에 열 추가
1. '1996_1997_debut.csv 파일 읽기

In [62]:
idol_list = pd.read_csv('1996_1997_debut.csv',encoding = 'euc-kr')
idol_list.shape

(10, 7)

2. 리스트 칼럼 확인

In [17]:
idol_list.columns

Index(['artist', 'artist_eng', 'member_num', 'leader_name', 'gender', 'agency',
       'idol_id'],
      dtype='object')

## 성별 데이터
3. 최종 수집 결과에 성별이 영어로 표기되어 있으므로 바꿔주기

In [18]:
add_gender = []
add_gender = idol_list.gender
add_gender

0    남
1    남
2    남
3    남
4    남
5    남
6    남
7    남
8    남
9    여
Name: gender, dtype: object

In [19]:
# 성별 영어로 바꿔주기

for i in range(len(add_gender)):
    if(add_gender[i] == '남'):
        add_gender[i] = 'male'
    else:
        add_gender[i] = 'female'

add_gender

/Users/mac/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/mac/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


0      male
1      male
2      male
3      male
4      male
5      male
6      male
7      male
8      male
9    female
Name: gender, dtype: object

## 소속사 데이터

In [20]:
add_agency = []
add_agency = idol_list.agency
add_agency

0         레오뮤직
1       뮤직 팩토리
2      DSP 미디어
3        비온디크루
4          매스컴
5       뮤직 팩토리
6       베토벤 뮤직
7            x
8    YG 엔터테인먼트
9         희레코드
Name: agency, dtype: object

### 크롤링 데이터에 성별, 소속사 추가하기
**성별과 소속사 부분은 각자 년도별 아이돌에 맞게 코드를 수정해서 하셔야 합니다**

In [63]:
test_list = [title, release_date, song_genre, creator, lyrics]
column_names = ['년도','가수','제목','장르','작사가', '가사']
column_v = [release_date, artist, title, song_genre, creator, lyrics]

test_df = pd.DataFrame(column_v)


In [64]:
test_p = test_df.head(6).T
test_p.head()


,0,1,2,3,4,5
0,19990801,언타이틀,떠나가지 마세요,댄스,,
1,19990305,언타이틀,학교 (Main Title),-,,다시 바꿔 생각해 너무 억울한걸\n다른 위치라면은 누구나 같은데\n학교폭력에 숨진 ...
2,19990201,언타이틀,떠나가지 마세요,댄스,유건형\n작사\n서정환\n작사\n유건형\n작곡\n유건형\n편곡,난 울고 있어요 그대도 아시나요\n미치도록 아픈 나의 가슴을\n그대 날 떠난 후엔 ...
3,19990201,언타이틀,나의 슬픈 사랑이야기,댄스,유건형\n작사\n유건형\n작곡\n유건형\n편곡,이젠 늦었어요 이미 그녀는\n다른 곳에 있죠 사랑한다는\n그 말 한마디 못한 채\n...
4,19990201,언타이틀,사랑연가,댄스,유건형\n작사\n유건형\n작곡\n유건형\n편곡,누굴 좋아해야 하나요\n누굴 사랑해야 하나요\n누굴 향해 고백하나요\n난 정말 알 ...


In [65]:
test_p.columns = column_names
test_p.tail(350)

,년도,가수,제목,장르,작사가,가사
0,19990801,언타이틀,떠나가지 마세요,댄스,,
1,19990305,언타이틀,학교 (Main Title),-,,다시 바꿔 생각해 너무 억울한걸\n다른 위치라면은 누구나 같은데\n학교폭력에 숨진 ...
2,19990201,언타이틀,떠나가지 마세요,댄스,유건형\n작사\n서정환\n작사\n유건형\n작곡\n유건형\n편곡,난 울고 있어요 그대도 아시나요\n미치도록 아픈 나의 가슴을\n그대 날 떠난 후엔 ...
3,19990201,언타이틀,나의 슬픈 사랑이야기,댄스,유건형\n작사\n유건형\n작곡\n유건형\n편곡,이젠 늦었어요 이미 그녀는\n다른 곳에 있죠 사랑한다는\n그 말 한마디 못한 채\n...
4,19990201,언타이틀,사랑연가,댄스,유건형\n작사\n유건형\n작곡\n유건형\n편곡,누굴 좋아해야 하나요\n누굴 사랑해야 하나요\n누굴 향해 고백하나요\n난 정말 알 ...
...,...,...,...,...,...,...
73,19970701,T.N.B,비키니-Ⅰ,댄스,이승호\n작사,계절중에 언제가 제일\n좋냐고 내게 물으면\n아물론 두말하면 잔소리\n여름 아니겠어...
74,19970701,T.N.B,노코멘트 (No Comment),댄스,이정은\n작사,외로워서 만났던 중에\n착각한 여자가 있었어\n사랑이란 말 해달라고\n매번 요구했지...
75,19970701,None,None,None,None,None
76,19970701,None,None,None,None,None


In [25]:
c_artist = []
c_artist = test_p.가수

1. 성별 추가하기

In [26]:
tmp_artist = []
tmp_artist = idol_list.artist

In [27]:
a_gender = []

for i in range(len(c_artist)) :
    if( c_artist[i] == tmp_artist[8] ):
        a_gender.append(add_gender[9])
    else :
        a_gender.append(add_gender[0])
#a_gender

2. 소속사 추가하기

In [28]:
a_agency = []

for i in range(len(c_artist)) :
    if( c_artist[i] == tmp_artist[0] ):
        a_agency.append(add_agency[0])
    elif( c_artist[i] == tmp_artist[1] ):
        a_agency.append(add_agency[1])
    elif( c_artist[i] == tmp_artist[2] ):
        a_agency.append(add_agency[2])
    elif( c_artist[i] == tmp_artist[3] ):
        a_agency.append(add_agency[3])
    elif( c_artist[i] == tmp_artist[4] ):
        a_agency.append(add_agency[4])
    elif( c_artist[i] == tmp_artist[5] ):
        a_agency.append(add_agency[5])
    elif( c_artist[i] == tmp_artist[6] ):
        a_agency.append(add_agency[6])
    elif( c_artist[i] == tmp_artist[7] ):
        a_agency.append(add_agency[7])
    elif( c_artist[i] == tmp_artist[8] ):
        a_agency.append(add_agency[8])
    else :
        a_agency.append(add_agency[9])
#a_agency

### 작사가 작곡가 나누기
1. 시험삼아 1개만 일단 해보기

In [29]:
creator[2]

'유건형\n작사\n서정환\n작사\n유건형\n작곡\n유건형\n편곡'

In [30]:
arr = []
tmp_arr = []
tmp = ""
lyricist = []
composer = []

l_idx = 0

arr = creator[2].split('작사')
print("arr : ", arr)
for i in range(len(arr)):
    if( '작곡' in arr[i]): 
        tmp_arr = arr[i].split('작곡')
        #print("tmp_arr:", tmp_arr)
        composer.append(tmp_arr[0].replace("\n",""))
        
    else:
        if( l_idx == 0): 
            lyricist.append(arr[i].replace("\n",""))   
            l_idx += 1
        else :
            lyricist[l_idx-1] += ','
            lyricist[l_idx-1] += arr[i].replace("\n","")
 
print("작곡가: " ,composer)
print("작사가: ", lyricist[0])

arr :  ['유건형\n', '\n서정환\n', '\n유건형\n작곡\n유건형\n편곡']
작곡가:  ['유건형']
작사가:  유건형,서정환


2. 전체 적용

In [51]:
add_lyricist = []
add_composer = []
arr = []
tmp_arr = []
tmp = ""



#print(creator)
for j in range(len(creator)):
    if( creator[j] == ""):
        add_lyricist.append("")
        add_composer.append("")
        continue
    arr = creator[j].split('작사')
    #print(j,":",arr)
    l_idx = 0
    for i in range(len(arr)):
        if( '작곡' in arr[i]): 
            tmp_arr = arr[i].split('작곡')
            #print("tmp_arr:", tmp_arr)
            add_composer.append(tmp_arr[0].replace("\n",""))
        else:

            if( l_idx == 0) :
                add_lyricist.append(arr[i].replace("\n",""))   
                l_idx += 1
            else :
                l_idx = len(add_lyricist)
                print(len(add_lyricist))
                add_lyricist[l_idx-1] += ','
                add_lyricist[l_idx-1] += arr[i].replace("\n","")
             
#print("작사가: ",add_lyricist) 
#print("작곡가: " ,add_composer)


3
7
10
11
71
72
73
74


### 발매년도에 .빼기

In [52]:
for i in range(len(release_date)):
    release_date[i] = release_date[i].replace(".","")
    
#release_date

### 가사에 "" '\n' 빼기

In [80]:
tmp = ""
for i in range(len(lyrics)):
    tmp = lyrics[i].replace("\n"," ")
    tmp = tmp.replace("'","")
    lyrics[i] = tmp.replace("\"","")
    
lyrics

['',
 '다시 바꿔 생각해 너무 억울한걸 다른 위치라면은 누구나 같은데 학교폭력에 숨진 인격들 그 생각들 집어쳐 어릴적 꿈 같은건 찾기가 힘든데 이겨낼 수 없는건 바로 우리 현실 그렇지만은 포기하지마 모든 걸 잃기전에 조금씩 바꿔 변해가는 이 세상 아름다운 우리 미래를 닫힌 가슴들을 열고서 함께 달려가  다시바꿔 생각해 너무 억울한걸 다른 위치라면은 누구나 같은데 학교폭력에 숨진 인격들 그 생각들 집어쳐 어릴적 꿈 같은건 찾기가 힘든데 이겨낼 수 없는건 바로 우리 현실 그렇지만은 포기하지마 모든 걸 잃기전에 조금씩 바꿔 변해가는 이 세상 아름다운 우리 미래를 닫힌 가슴들을 열고서 함께 달려가 이른 아침 눈 뜨자 마자 발길 옮겨 서둘러 도착해 교실메꿔 특별한 꿈들 펼치며 모두 굽히지 않는 또 투지로 덤벼 날 괴롭힌 어느 누군가 다른 누굴 괴롭히려 애쓰나 다가온 순간 함께 나눈 많은 시간 이겨 나가봐',
 '난 울고 있어요 그대도 아시나요 미치도록 아픈 나의 가슴을 그대 날 떠난 후엔 세상 그 무엇들도 내 맘 달래줄 수가 없었어요 그토록 나 그댈 사랑했던 추억들이 날 괴롭혀요 그보다 더 나 슬퍼지는건 지쳐가는 나였었기에 그대 제발 떠나가지 마세요 이렇게 헤어지나요 진실한 내 사랑을 맘속 깊이 간직한 채 훔쳐 멀리 도망쳐간 내 맘 앗아간 난 망가질 판 망설임조차 없이도 난 버려지고 쓰러져 꺾였고 또 날 몰아쳤었던 힘들던 힘겹던 꿈 같았었던 시련 내 모든 것을 다 바쳐 내 가슴 가득 상처  난 초라해져요 그대도 아시나요 끝도 없이 흐르는 내 눈물을 왜 날 떠나갔는지 쉽게 이별하는지 찢어지는 가슴 너무 아파요 그토록 나 그댈 사랑했던 추억들이 날 괴롭혀요 그보다 더 나 슬퍼지는건 지쳐가는 나였었기에 그대 제발 떠나가지 마세요 이렇게 헤어지나요 진실한 내 사랑을 맘속 깊이 간직한 채  그대여 날 떠나가나요 난 그대 없인 정말 견딜 수 없어 이렇게 애원할게 내 곁에 돌아와줘요 자꾸마구자꾸 뭉쳐 가슴 가득 꽉찬 입가로 흐르는 눈물 닦아줄듯한 괴롭게 떠나갔고 외롭던 아픔들

### 안타깝게도 최고순위는 직접...ㅎ..

In [81]:
rank = []

In [82]:

column_names = ['년도','가수','제목','남녀','장르','최고순위','작사가','작곡가','소속사' ,'가사']
column_v = [release_date, artist, title, a_gender, song_genre,rank,add_lyricist,add_composer,a_agency, lyrics]
test_df = pd.DataFrame(column_v)


In [83]:
test_p = test_df.head(10).T
test_p.head()


,0,1,2,3,4,5,6,7,8,9
0,19990801,언타이틀,떠나가지 마세요,male,댄스,None,,,레오뮤직,
1,19990305,언타이틀,학교 (Main Title),male,-,None,,,레오뮤직,다시 바꿔 생각해 너무 억울한걸 다른 위치라면은 누구나 같은데 학교폭력에 숨진 인격...
2,19990201,언타이틀,떠나가지 마세요,male,댄스,None,"유건형,서정환",유건형,레오뮤직,난 울고 있어요 그대도 아시나요 미치도록 아픈 나의 가슴을 그대 날 떠난 후엔 세상...
3,19990201,언타이틀,나의 슬픈 사랑이야기,male,댄스,None,유건형,유건형,레오뮤직,이젠 늦었어요 이미 그녀는 다른 곳에 있죠 사랑한다는 그 말 한마디 못한 채 난 이...
4,19990201,언타이틀,사랑연가,male,댄스,None,유건형,유건형,레오뮤직,누굴 좋아해야 하나요 누굴 사랑해야 하나요 누굴 향해 고백하나요 난 정말 알 수가 ...


In [84]:
test_p.columns = column_names
test_p.tail(350)

,년도,가수,제목,남녀,장르,최고순위,작사가,작곡가,소속사,가사
0,19990801,언타이틀,떠나가지 마세요,male,댄스,None,,,레오뮤직,
1,19990305,언타이틀,학교 (Main Title),male,-,None,,,레오뮤직,다시 바꿔 생각해 너무 억울한걸 다른 위치라면은 누구나 같은데 학교폭력에 숨진 인격...
2,19990201,언타이틀,떠나가지 마세요,male,댄스,None,"유건형,서정환",유건형,레오뮤직,난 울고 있어요 그대도 아시나요 미치도록 아픈 나의 가슴을 그대 날 떠난 후엔 세상...
3,19990201,언타이틀,나의 슬픈 사랑이야기,male,댄스,None,유건형,유건형,레오뮤직,이젠 늦었어요 이미 그녀는 다른 곳에 있죠 사랑한다는 그 말 한마디 못한 채 난 이...
4,19990201,언타이틀,사랑연가,male,댄스,None,유건형,유건형,레오뮤직,누굴 좋아해야 하나요 누굴 사랑해야 하나요 누굴 향해 고백하나요 난 정말 알 수가 ...
...,...,...,...,...,...,...,...,...,...,...
73,19970701,T.N.B,비키니-Ⅰ,male,댄스,None,"이정은,",None,x,계절중에 언제가 제일 좋냐고 내게 물으면 아물론 두말하면 잔소리 여름 아니겠어 수영...
74,19970701,T.N.B,노코멘트 (No Comment),male,댄스,None,None,None,x,외로워서 만났던 중에 착각한 여자가 있었어 사랑이란 말 해달라고 매번 요구했지만 난...
75,19970701,None,None,male,None,None,None,None,희레코드,None
76,19970701,None,None,male,None,None,None,None,희레코드,None


In [85]:
test_p.shape

(78, 10)

In [89]:
# 곡 전처리
test_p['제목'].isnull().sum()

3

In [87]:
test_p.tail()

,년도,가수,제목,남녀,장르,최고순위,작사가,작곡가,소속사,가사
73,19970701,T.N.B,비키니-Ⅰ,male,댄스,None,"이정은,",None,x,계절중에 언제가 제일 좋냐고 내게 물으면 아물론 두말하면 잔소리 여름 아니겠어 수영...
74,19970701,T.N.B,노코멘트 (No Comment),male,댄스,None,None,None,x,외로워서 만났던 중에 착각한 여자가 있었어 사랑이란 말 해달라고 매번 요구했지만 난...
75,19970701,None,None,male,None,None,None,None,희레코드,None
76,19970701,None,None,male,None,None,None,None,희레코드,None
77,,None,None,male,None,None,None,None,희레코드,None


In [88]:
test_p.to_csv('k_pop_3.tsv',index=False,header=None,sep='\t') #tsv파일로 저장